### Set standard imports

In [ ]:
import matplotlib,lmfit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib tk
#for more comfort use
#%matplotlib qt
import KiMoPack.plot_func as pf
from importlib import reload
reload(pf)

### Load and inspect Data 

In [ ]:
ta=pf.TA('XES_on.SIA',path='Data',sep=',',units='eV',sort_indexes=True,decimal='.',data_type='X-ray emission intensity',baseunit='ps')   #type filename and path (leave path empty if in the same folder)
#ta=pf.TA('XES_diff.SIA',path='Data',sep=',',units='eV',sort_indexes=True,decimal='.',data_type='X-ray emission differential')
#ta=pf.TA("sample_1.hdf5",path="Data")  #reload saved projects using the same syntax
#ta=pf.TA('gui')                       #use a GUI to select data

### Shape data and plot Raw images

In [ ]:
plt.close('all')
ta.rel_wave=np.arange(7053,7061,1)      # use to manually select interesting wavelength for the kinetics
ta.wavelength_bin=1                                        # width of wavelength bin in kinetics
ta.rel_time=[-0.5,-0.1,0,0.1,0.3,0.5,1,1.5,3,5,10,15,20]      # use to manually set interesting times for the spectra
ta.time_width_percent=0                                    # rebinning of time_points in percent
ta.timelimits=[-1,200]                                     # use to manually limit the range to plot (and fit)     
ta.log_scale=False                                         # use to plot the 2d plots with logarithmic intensity scale
ta.bordercut=[7045,7065]                                      # use to set the outer wavelength limits
#ta.scattercut=[522,605]                                     # this region is blanked out to block scatter, this can be a list of regions
ta.intensity_range=[0,7]                           # set a value 5e-3 or a range [-1e-3,5e-3] for 2d plotting
ta.Plot_RAW()                                      # Plot Raw Spectra

## Apply some gentle smoothing?

In [ ]:
if 1:
    ta.ds=pf.Frame_golay(ta.ds,7,3)
    ta.Plot_RAW() 

### Perform fast fitting using independent exponential decay (first order)

In [ ]:
import function_library as func
reload(func)

In [ ]:
plt.close('all')                                             # Close old spectra
ta.mod=func.P12                                        # Choose the model 'exponential', 'consecutive' and 'fast_consecutive'are build in
par=lmfit.Parameters()                                       # create empty parameter object

par.add('k0',value=1/0.275,vary=True)                                # Add second rate component
par.add('k1',value=1/7.2,vary=True)                                # Add one rate component
par.add('k2',value=1/0.45,vary=True)                                 # Add second rate component
par.add('t0',value=-0.0167,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust (for all)
par.add('resolution',value=0.0935,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (for all)
par.add('explicit_GS')
#par.add('infinite')                                                # Keyword for an non decaying component
par.add('background')                                             # Keyword to fit the Background Disable the background substraction in the loading cell for best effect
ta.log_fit=False                                               # fit in linear or log scale
if 0:
    for key in par.keys():
        par[key].vary=False
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting
ta.error_matrix_amplification=10;                              # Choose how much stronger should error be plotted                                                    # 2D plots in linear or log scale
ta.Plot_fit_output()                                 # plot the fit output

### Calculate the absolute spectra

In [ ]:
for col in ta.re['DAC'].columns:
    ta.re['DAC'].loc[:,col]+=ta.re['DAC'].background
ta.re['DAC']=ta.re['DAC'].drop('background',axis=1)
ta.Plot_fit_output(0)

In [ ]:
ta.Save_Powerpoint()